# 多格式文件轉換 (Multi-format Conversion)

Docling 支援自動偵測多種輸入文件格式，並轉換成適合 LLM 使用的輸出格式。  
完整的支援清單可參考官方「Supported Input and Output Formats」。

你可以透過 `DocumentConverter` 類別的 `allowed_formats` 參數來限制允許的輸入格式，  
未包含在此列表的格式會被忽略。

同時，也可以利用 Python 的 `Path.glob()` 方法自行篩選想要轉換的檔案。


In [1]:
# 範例：限制允許的輸入格式
from docling.document_converter import DocumentConverter, InputFormat

doc_converter = DocumentConverter(
    allowed_formats=[
        InputFormat.PDF,
        InputFormat.IMAGE,
        InputFormat.DOCX,
        InputFormat.HTML,
        InputFormat.PPTX,
        InputFormat.ASCIIDOC,
        InputFormat.CSV,
        InputFormat.MD,
    ]
)


## 自行過濾要處理的檔案類型

如果不想用 `allowed_formats`，可以使用 Python 的 `glob()` 方法指定副檔名過濾。


In [3]:
from pathlib import Path

input_path = Path("/work/genai-apps/dataprep/sample-data/multi-format")

# 自訂允許的副檔名
allowed_extensions = ["*.pdf", "*.docx", "*.xlsx", "*.pptx", "*.csv", "*.asciidoc"]

input_doc_paths = []
for ext in allowed_extensions:
    input_doc_paths.extend(input_path.glob(ext))

print(f"找到 {len(input_doc_paths)} 個檔案可供轉換")
for p in input_doc_paths[:5]:
    print(" -", p.name)


找到 6 個檔案可供轉換
 - amt_handbook_sample.pdf
 - word_sample.docx
 - excel-test-01.xlsx
 - powerpoint_with_image.pptx
 - csv-comma.csv


## 建立輸出資料夾並設定輸出路徑


In [19]:
OUTPUT_DIR = Path("/work/tmp/multi-converted")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)


## 使用 `convert_all` 進行批次轉換

Docling 的 `convert_all` 方法可將所有檔案一次轉成 `DoclingDocument` 物件，  
後續可再匯出成各種格式（如 JSON、Markdown）。


```python
from docling.models.docling_document import ImageRefMode

conv_results = doc_converter.convert_all(
    input_doc_paths,
    raises_on_error=False,  # 發生錯誤時不中斷整批
)
len(conv_results)


## 將轉換結果輸出為 JSON

這裡示範將每個 `DoclingDocument` 輸出成 `.json` 格式，  
你可以在 VS Code 或其他編輯器中查看結果。


```python
saved = []
for conv_res in conv_results:
    if conv_res.document is None:
        print(f"[Skip] {conv_res.input_path.name}")
        continue

    out_file = OUTPUT_DIR / f"{conv_res.input_path.stem}.json"
    conv_res.document.save_as_json(out_file, image_mode=ImageRefMode.PLACEHOLDER)
    saved.append(out_file)

print(f"完成輸出 {len(saved)} 個 JSON 檔案")
for p in saved[:5]:
    print(" -", p)


## 實際執行預寫好的腳本

In [13]:
!python3 /work/genai-apps/dataprep/multi-format.py

[INFO] Found 6 files in /work/genai-apps/dataprep/sample-data/multi-format
2025-10-07 09:09:43,545 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-10-07 09:09:43,551 - INFO - Going to convert document batch...
2025-10-07 09:09:43,551 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 58627ba972cff869acb44714af27a59e
2025-10-07 09:09:43,553 - INFO - Loading plugin 'docling_defaults'
2025-10-07 09:09:43,553 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-10-07 09:09:43,555 - INFO - Loading plugin 'docling_defaults'
2025-10-07 09:09:43,556 - INFO - Registered ocr engines: ['easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-10-07 09:09:43,558 - INFO - Accelerator device: 'cpu'
2025-10-07 09:09:43,560 - INFO - Accelerator device: 'cpu'
2025-10-07 09:09:44,377 - INFO - Accelerator device: 'cpu'
2025-10-07 09:09:44,531 - INFO - Processing document amt_handbook_sample.pdf
2025-10-07 09:09:45,607 - INFO - Finished converting document a

## 驗證輸出結果

到 `/tmp/multi-converted` 資料夾查看，應該可以看到與輸入檔案同名的 `.json` 檔。  
可打開檢視層級結構：標題、段落、表格等會被轉成對應的 JSON 層級。


In [20]:
from pathlib import Path

OUT_PATH = Path(OUTPUT_DIR)

for p in sorted(OUT_PATH.glob("*.json"))[:10]:
    print(p.name)

print("總數:", len(list(OUT_PATH.glob("*.json"))))



adoc_test_01.json
amt_handbook_sample.json
csv-comma.json
excel-test-01.json
powerpoint_with_image.json
word_sample.json
總數: 6


### 小提醒
- `allowed_formats` 可以快速限制處理的輸入格式；若要更彈性，可用 Python 的 `glob()` 自行篩選副檔名。
- 輸出除了 JSON，也可使用 `.save_as_markdown()`、`.save_as_html()` 等方法。
- 若文件包含掃描影像且需 OCR，請確保環境已安裝 OCR 引擎（如 Tesseract/EasyOCR）。
